##Other

### OS cd

In [2]:
import os
os.chdir("/content/drive/MyDrive/Ntu/DS/HW6/my_codes")
print(os.getcwd())

/content/drive/MyDrive/Ntu/DS/HW6/my_codes


###Module of main

In [5]:
import os
import pandas as pd

from utils.options import args
import utils.common as utils

from importlib import import_module

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR as MultiStepLR
from torch.optim.lr_scheduler import StepLR as StepLR

from data import dataPreparer

import warnings, math

Gobal of main

In [6]:
warnings.filterwarnings("ignore")

device = torch.device(f"cuda:{args.gpus[0]}")

checkpoint = utils.checkpoint(args)

### Data loading...

In [7]:
# Data loading
print('=> Preparing data..')

# data loader

loader = dataPreparer.Data(args,data_path=args.src_data_path,label_path=args.src_label_path)

data_loader = loader.loader_train
data_loader_eval = loader.loader_test

=> Preparing data..


## Main def

In [ ]:
def main():

    start_epoch = 0
    best_acc = 0.0
 
    

    
    # Create model
    print('=> Building model...')

    # load training model
    model = import_module(f'model.{args.arch}').__dict__[args.model]().to(device)
    

    # Load pretrained weights
    if args.pretrained:
 
        ckpt = torch.load(args.source_dir + args.source_file, map_location = device)
        state_dict = ckpt['state_dict']

        model.load_state_dict(state_dict)
        model = model.to(device)
        
    if args.inference_only:
        acc = inference(args, data_loader_eval, model, args.output_file)
        print(f'Test acc {acc:.3f}\n')
        return

    param = [param for name, param in model.named_parameters()]
    
    optimizer = optim.SGD(param, lr = args.lr, momentum = args.momentum, weight_decay = args.weight_decay)
    scheduler = StepLR(optimizer, args.lr_decay_step, gamma = args.lr_gamma)


    for epoch in range(start_epoch, args.num_epochs):
        print(f"**epoch:{epoch+1}/{args.num_epochs}**", flush=True)  ## record
        scheduler.step(epoch)
        
        train(args, data_loader, model, optimizer, epoch)
        
        test_acc = test(args, data_loader_eval, model)
   
        is_best = best_acc < test_acc
        best_acc = max(best_acc, test_acc)
        

        state = {
            'state_dict': model.state_dict(),
            
            'optimizer': optimizer.state_dict(),
            
            'scheduler': scheduler.state_dict(),
            
            'epoch': epoch + 1
        }
        checkpoint.save_model(state, epoch + 1, is_best)
        
    inference(args, data_loader_eval, model, args.output_file)
    
    print(f'Best acc: {best_acc:.3f}\n')

## Train
per epoch

In [ ]:
def train(args, data_loader, model, optimizer, epoch):
    losses = utils.AverageMeter()

    acc = utils.AverageMeter()

    criterion = nn.CrossEntropyLoss()
    
    num_iterations = len(data_loader)
    
    # switch to train mode
    model.train()

    
         
    for i, (inputs, targets, _) in enumerate(data_loader, 1):
        print(f"epoch:{epoch+1}/{args.num_epochs}, iter:{i}/{num_iterations}", flush=True)  ## record
        num_iters = num_iterations * epoch + i

        inputs = inputs.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        
        # train
        output = model(inputs)
        loss = criterion(output, targets)

        # optimize cnn
        loss.backward()
        optimizer.step()

        ## train weights        
        losses.update(loss.item(), inputs.size(0))
        
        ## evaluate
        prec1, _ = utils.accuracy(output, targets, topk = (1, 5))
        acc.update(prec1[0], inputs.size(0))

        
        if i % args.print_freq == 0:     
            print(
                'Epoch[{0}]({1}/{2}): \n'
                'Train_loss: {train_loss.val:.4f} ({train_loss.avg:.4f})\n'
                'Train acc {acc.val:.3f} ({acc.avg:.3f})\n'.format(
                epoch, i, num_iterations, 
                train_loss = losses,
                acc = acc))

## Test (Validation)

In [19]:
def test(args, loader_test, model):
    losses = utils.AverageMeter()
    acc = utils.AverageMeter()

    criterion = nn.CrossEntropyLoss()

    num_iterations_test = len(loader_test) ## record


    # switch to eval mode
    model.eval()

    with torch.no_grad():
        for i, (inputs, targets, datafile) in enumerate(loader_test, 1):
            print(f"Eval-- iter:{i}/{num_iterations_test}", flush=True) ## record
            
            inputs = inputs.to(device)
            targets = targets.to(device)
             
            preds = model(inputs)
            loss = criterion(preds, targets)
        
            # image classification results
            prec1, _ = utils.accuracy(preds, targets, topk = (1, 5))
            losses.update(loss.item(), inputs.size(0))
            acc.update(prec1[0], inputs.size(0))

 
    print(f'Test acc {acc.avg:.3f}\n')

    return acc.avg

## Inference

In [ ]:
def inference(args, loader_test, model, output_file_name):
    losses = utils.AverageMeter()
    acc = utils.AverageMeter()

    criterion = nn.CrossEntropyLoss()
    outputs = []
    datafiles = []
    count = 1
    # switch to eval mode
    model.eval()

    with torch.no_grad():
        for i, (inputs, targets, datafile) in enumerate(loader_test, 1):
            
            inputs = inputs.to(device)
            targets = targets.to(device)
            
          
            preds = model(inputs)
    
            loss = criterion(preds, targets)
        
            # image classification results
            prec1, _ = utils.accuracy(preds, targets, topk = (1, 5))
            losses.update(loss.item(), inputs.size(0))
            acc.update(prec1[0], inputs.size(0))
            
            _, output = preds.topk(1, 1, True, True)
            
            outputs.extend(list(output.reshape(-1).cpu().detach().numpy()))
            
            datafiles.extend(list(datafile))
            
            count += inputs.size(0)
    

    output_file = dict()
    output_file['image_name'] = datafiles
    output_file['label'] = outputs
    
    output_file = pd.DataFrame.from_dict(output_file)
    output_file.to_csv(output_file_name, index = False)
    
    return acc.avg

## Main and control panel

control

In [18]:
args

5

main

In [12]:
if __name__ == '__main__':
    main()

串流輸出內容已截斷至最後 5000 行。
epoch:7/10, iter:562/704
epoch:7/10, iter:563/704
epoch:7/10, iter:564/704
epoch:7/10, iter:565/704
Epoch[6](565/704): 
Train_loss: 0.6320 (0.5330)
Train acc 81.250 (85.689)

epoch:7/10, iter:566/704
epoch:7/10, iter:567/704
epoch:7/10, iter:568/704
epoch:7/10, iter:569/704
epoch:7/10, iter:570/704
Epoch[6](570/704): 
Train_loss: 0.4211 (0.5326)
Train acc 90.625 (85.702)

epoch:7/10, iter:571/704
epoch:7/10, iter:572/704
epoch:7/10, iter:573/704
epoch:7/10, iter:574/704
epoch:7/10, iter:575/704
Epoch[6](575/704): 
Train_loss: 0.6434 (0.5325)
Train acc 79.688 (85.696)

epoch:7/10, iter:576/704
epoch:7/10, iter:577/704
epoch:7/10, iter:578/704
epoch:7/10, iter:579/704
epoch:7/10, iter:580/704
Epoch[6](580/704): 
Train_loss: 0.3356 (0.5319)
Train acc 92.188 (85.722)

epoch:7/10, iter:581/704
epoch:7/10, iter:582/704
epoch:7/10, iter:583/704
epoch:7/10, iter:584/704
epoch:7/10, iter:585/704
Epoch[6](585/704): 
Train_loss: 0.5149 (0.5319)
Train acc 90.625 (85.724)

epoc